### Workflow in preprocessing LIS output to construct a water balance for GRACE comparison and forward modeling.

by Anthony Arendt and Landung Setiawan

In [1]:
%matplotlib inline
import os
import sys
import datetime
import glob
import re
from collections import OrderedDict

import xarray as xr
print('Xarray version: {}'.format(xr.__version__))

import pandas as pd
print('Pandas version: {}'.format(pd.__version__))

from dask.diagnostics import ProgressBar

import seaborn as sb

import matplotlib
print('Matplotlib version: {}'.format(matplotlib.__version__))
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

from himatpy.LIS import utils as LISutils

import cartopy
print('Cartopy version: {}'.format(cartopy.__version__))
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

Xarray version: 0.9.6
Pandas version: 0.20.3
Matplotlib version: 2.1.0
Cartopy version: 0.15.1


## Serializing LIS to yearly blocks single netCDF, retaining the daily resolution. 

Only certain variables are kept and unit adjustments are performed

In [2]:
variables = ['Qsm_tavg','Rainf_tavg','Qs_tavg','Snowf_tavg','Qsb_tavg','Evap_tavg','TWS_tavg']

The function `process_lis_data` builds of other function within `himatpy.LIS.utils` library to process the data.

In [3]:
print(LISutils.process_lis_data.__doc__)


    This function reads daily LIS output, selects a subset of variables, and serializes to NetCDF files 
    with daily resolution and yearly span.
    
    Parameters
    ----------
    data_dir : String.
        The location of the Raw LIS NetCDF data
    nc_path : String.
        The location of the output NetCDF.
    startyear: Integer.
        The year to start processing.
    endyear: Integer.
        The year to end processing.
    **kwargs: Other keyword arguments associated with get_xr_dataset
    
    Returns
    -------
    None
    


**Note: There is a script that will use the `process_lis_data` function above called `LISpreprocess.py`**

```bash
$python himatpy/LIS/LISpreprocess.py --help
usage: LISpreprocess.py [-h] [--startyear STARTYEAR] [--endyear ENDYEAR] [-v] <LISDIR> <OUTDIR>

positional arguments:
  <LISDIR>              The directory of Raw LIS NetCDF
  <OUTDIR>              The directory for the preprocessed data

optional arguments:
  -h, --help            show this help message and exit
  --startyear STARTYEAR
                        Specify start year to process. Ex. 2008
  --endyear ENDYEAR     Specify end year to process. Ex. 2004
  -v, --version         show program's version number and exit
```

Once the script successfully run, there should be yearly, daily resolution netCDF files.

In [4]:
yearlydir = '/att/nobackup/lsetiawa/LISYearly/'

In [5]:
ds = xr.open_mfdataset(os.path.join(yearlydir, '*.nc'), chunks={'time': 1})

In [6]:
ds.info()

xarray.Dataset {
dimensions:
	east_west = 1896 ;
	north_south = 1696 ;
	time = 5812 ;

variables:
	float64 longitude(north_south, east_west) ;
	float64 latitude(north_south, east_west) ;
	datetime64[ns] time(time) ;
		time:long_name = time ;
		time:time_increment = 86400 ;
		time:begin_time = 000000 ;
	float64 Qsm_tavg(time, north_south, east_west) ;
		Qsm_tavg:units = mm we ;
		Qsm_tavg:standard_name = snowmelt ;
		Qsm_tavg:long_name = Daily snowmelt in units of mm we ;
		Qsm_tavg:vmin = 0.0 ;
		Qsm_tavg:vmax = 0.0 ;
	float64 Rainf_tavg(time, north_south, east_west) ;
		Rainf_tavg:units = mm we ;
		Rainf_tavg:standard_name = precipitation_rate ;
		Rainf_tavg:long_name = Daily precipitation_rate in units of mm we ;
		Rainf_tavg:vmin = 0.0 ;
		Rainf_tavg:vmax = 0.0 ;
	float64 Qs_tavg(time, north_south, east_west) ;
		Qs_tavg:units = mm we ;
		Qs_tavg:standard_name = surface_runoff_amount ;
		Qs_tavg:long_name = Daily surface_runoff_amount in units of mm we ;
		Qs_tavg:vmin = 0.0 ;
		Qs_

In [7]:
print('Time span: {:%Y-%m-%d} to {:%Y-%m-%d}'.format(pd.to_datetime(ds.time.values.min()), 
                                                     pd.to_datetime(ds.time.values.max())))

Time span: 2001-01-02 to 2016-12-30


## Calculating montly average from yearly, daily resolution netCDF.

In [8]:
# Location to save the output of the monthly average
out_pth = '/att/nobackup/lsetiawa/'

In [9]:
monthlyds = LISutils.get_monthly_avg(ds, export_nc=True, out_pth=out_pth)

Exporting /att/nobackup/lsetiawa/LISMonthly.nc
[########################################] | 100% Completed | 32min 14.0s


In [10]:
# Checking the netCDF File quickly
dsout = xr.open_dataset(os.path.join(out_pth, 'LISMonthly.nc'))

In [11]:
dsout.info()

xarray.Dataset {
dimensions:
	east_west = 1896 ;
	north_south = 1696 ;
	time = 192 ;

variables:
	float64 longitude(north_south, east_west) ;
	float64 latitude(north_south, east_west) ;
	datetime64[ns] time(time) ;
	float64 Qsm_tavg(time, north_south, east_west) ;
		Qsm_tavg:units = mm we ;
		Qsm_tavg:standard_name = snowmelt ;
		Qsm_tavg:long_name = Cumulative monthly snowmelt in units of mm we ;
		Qsm_tavg:vmin = 0.0 ;
		Qsm_tavg:vmax = 0.0 ;
	float64 Rainf_tavg(time, north_south, east_west) ;
		Rainf_tavg:units = mm we ;
		Rainf_tavg:standard_name = precipitation_rate ;
		Rainf_tavg:long_name = Cumulative monthly precipitation_rate in units of mm we ;
		Rainf_tavg:vmin = 0.0 ;
		Rainf_tavg:vmax = 0.0 ;
	float64 Qs_tavg(time, north_south, east_west) ;
		Qs_tavg:units = mm we ;
		Qs_tavg:standard_name = surface_runoff_amount ;
		Qs_tavg:long_name = Cumulative monthly surface_runoff_amount in units of mm we ;
		Qs_tavg:vmin = 0.0 ;
		Qs_tavg:vmax = 0.0 ;
	float64 Snowf_tavg(time, north

### This workflow continues on another notebook [>>](../GRACE_MASCON/GRACE_LandInformationSystem.ipynb)